# Tutorial 01: Start with pyTENAX

Import all libraries needed for running basic TENAX

In [ ]:
from pathlib import Path
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Import pyTENAX
from pyTENAX import tenax, plotting

We need to specify path to directory with test data (RES_DIR)

In [ ]:
# Get the directory of tenax
SOURCE_DIR = Path(tenax.__file__).resolve()
HOME_DIR = SOURCE_DIR.parents[2]  # parents[2] is the third parent directory
RES_DIR = join(HOME_DIR, "res")
print(SOURCE_DIR )
print (HOME_DIR)
print ( RES_DIR)

Let's initiate TENAX class with given setup. For more information about TENAX class, please refer to API documention.

In [ ]:
# Initiate TENAX class with customized setup
S = tenax.TENAX(
    return_period=[
        2,
        5,
        10,
        20,
        50,
        100,
        200,
    ],
    durations=[10, 60, 180, 360, 720, 1440], #durations are in minutes and they refer to depth of rainfall within given duration
    time_resolution=5,  # time resolution in minutes
    left_censoring=[0, 0.90], # left censoring threshold 
    alpha=0.05, #dependence of shape on T depends on statistical significance at the alpha-level.
    min_rain = 0.1, #minimum rainfall depth threshold
)